# Fine-tune a pretrained model

本チュートリアルでは、事前学習済みモデルに対してファインチューニングを行う方法を学びます。
事前学習済みモデルに対して、特定のタスクに合わせたデータセットで追加の学習を行得ことで、ドメイン特化のモデルを得ることが期待できます。

本チュートリアルは、[Hugging Face Tranformers チュートリアル](https://huggingface.co/docs/transformers/v4.57.1/ja/training) を元に、一部加筆・修正して作成しています。

## Dependencies

このチュートリアルコードをすべて実行するためには、明示的に `import` するライブラリの他に必要なものは特にありません。

In [ ]:
# run this cell if you are working in google colab

%pip install evaluate

In [ ]:
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_scheduler,
    Trainer,
    TrainingArguments,
)
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

## Prepare a dataset

今回は、[google-bert/bert-base-cased](https://huggingface.co/google-bert/bert-base-cased) (BERT) という Masked Language Model (MLM) をファインチューンしてみます。
MLM とは、文章中の隠された (masked) 部分に当てはまる単語を推測するタスクを行う言語モデルです。
BERT は [BookCorpus](https://yknzhu.wixsite.com/mbweb) と呼ばれる、11308 冊の未出版書籍と、英語の Wikipedia によって事前学習が行われています。

今回は、BERT に対して、[yelp_review_full](https://huggingface.co/datasets/Yelp/yelp_review_full) というデータセットでファインチューニングを行います。
このデータセットは、飲食店や店舗のレビューサイトである Yelp 上のレビュー文章から構成されています。
早速、データセットをロードしましょう。

In [ ]:
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

データセットがロードできたら、トーカナイザをロードして事前処理を行いましょう。

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

実行時間短縮のために、データセットから適当な部分セットを作成できます。

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## Train

追加データセットの準備ができたので、ここから本格的にファインチューニングを行っていきます。
ファインチューニングを行う手法にはいくつかあり、代表的なものは以下の 3 通りです。

- `🤗 Transformers` が提供する `Trainer` クラスを利用するもの
- `Kelas` API を利用して `TensorFlow` で訓練するもの
- ネイティブの `PyTorch` で訓練するもの

これらの利点・欠点は下表のとおりです。

| method | advantage 👍 | disadvantage 👎 |
|: ---- |: ---- |: ---- |
| `Trainer` | 高水準 API を用いて短いコードで実装できる | カスタム性に欠ける |
| `Kelas` + `TensorFlow` | `TensorFlow` 専用の TPU ハードウェアや `Keras` の API が利用できる | `🤗 Transformers` はそもそも `PyTorch` 中心で、一部の機能は PyTorch 限定 |
| Native `PyTorch` | 低水準 API を用いて柔軟にカスタマイズできる | コード量が多く、実装が比較的複雑 |

本チュートリアルでは、`Trainer` と Native `PyTorch` の 2 通りの手法を紹介します。

### Train with PyTorch Trainer

`Trainer` クラスを用いたファインチューニングの手順を紹介します。
`🤗 Transformers` が提供する高水準 API を用いて、数行のプログラムで簡潔に記述することができます。

まずモデルをロードし、予想される (マスクされる) ラベルの数を指定します。

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

#### Training Hyperparameters

学習時のオプションとハイパーパラメータから構成される `TrainingArguments` クラスを作成します。
学習時のオプションとは、例えば学習後のパラメータファイルの保存先や、損失関数の値のログのタイミングなどを含みます。
また、ハイパーパラメータとは、学習率のスケジューラやエポック数などを含みます。
何も指定しなければ、デフォルトの値が利用されます。

```python
training_args = TrainingArguments(output_dir="test_trainer")
```

#### Evaluate

`Trainer` は、デフォルトではモデルのパフォーマンスを評価しません。
モデルのパフォーマンス評価を行うには、メトリクスを計算して報告する関数を `Trainer` に渡す必要があります。
`🤗 Evaluate` ライブラリでは、`evaluate.load` 関数を使用して読み込むことができる、`accuracy` 関数が用意されています。

In [ ]:
metric = evaluate.load("accuracy")

`metric.compute()` を呼ぶことで、予測精度を計算することができます。
なお、すべての `🤗 Transformers` モデルの出力は logit だそうです。

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

評価メトリクスをファインチューニング中に計算したい場合、学習引数 `eval_strategy` を利用できます。
今回は、各エポック終了時に計算するように設定します。

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

#### Trainer

モデル、学習引数、トレーニング/テストデータセット、評価メトリクスを指定して、`Trainer` オブジェクトを作成します。

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

`Trainer.train()` を実行して、ファインチューニングが行われます。

In [ ]:
trainer.train()

### Train in native Pytorch

次に、ネイティブ `PyTorch` でファイチューニングを行う手順を紹介します。
複雑ではあるものの、カスタマイズ性の高い学習ループを構成できます。
なお、以下の部分は上で定義した変数と衝突するため、`marimo` でチュートリアルを実行している方は、一度セッションを切って、`Trainer` API による実装部分をコメントアウトしてから、以下のプログラムを実行してください。

まずデータセットのロードを行うのですが、

1. モデルはトークン化前のオリジナルテキストを受け取らないので、`text` 列を削除する。
2. モデルは引数の名前を `labels` と期待しているので、`label` 列を `labels` に名前を変更しています。

In [ ]:
# run this cell if you are working on ipynb (google colab)

# del model
# del trainer
# torch.cuda.empty_cache()

In [ ]:
# tokenized_datasets_in_need = tokenized_datasets.remove_columns(["text"])
# tokenized_datasets_pt = tokenized_datasets_in_need.rename_column("label", "labels")
# tokenized_datasets_pt.set_format("torch")

# small_train_dataset_pt = tokenized_datasets_pt["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset_pt = tokenized_datasets_pt["test"].shuffle(seed=42).select(range(1000))

#### DataLoader

トレーニングデータセットとテストデータセット用の `DataLoader` を作成して、データのバッチをイテレータとして取り出せるようにします。

In [ ]:
# train_dataloader = DataLoader(small_train_dataset_pt, shuffle=True, batch_size=8)
# eval_dataloader = DataLoader(small_eval_dataset_pt, batch_size=8)

併せて、モデルのロードも行ってしまいます。
やり方は `Trainer` の場合と同じです。

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

#### Opeimizer and learning rate scheduler

オプティマイザと学習率スケジューラを作成します。
ここでは、`AdamW` を用いてモデルの最適化を行うことにします。

In [ ]:
# optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
# num_epochs = 3
# num_training_steps = num_epochs * len(train_dataloader)
# lr_scheduler = get_scheduler(
#     name="linear",
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps=num_training_steps,
# )

また、ファインチューニングを行うデバイスを指定しておきましょう。

In [ ]:
# for NVIDIA GPU (CUDA)
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# for Apple GPU (MPS)
# device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# model.to(device)

#### Train

学習の進捗を追跡するために、`tqdm` ライブラリを使用して進行状況バーを表示させます。

In [ ]:
# progress_bar = tqdm(range(num_training_steps))

# model.train()
# for epoch in range(num_epochs):
#     for batch_t in train_dataloader:
#         batch_train = {k: v.to(device) for k, v in batch_t.items()}
#         outputs_train = model(**batch_train)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)

#### Evaluate

`Trainer` の際と同様に、評価メトリックを導入します。
ここでは、各エポックの最後にメトリックを計算する代わりに、`add_batch` を使用してすべてのバッチを蓄積しておき、最後にメトリックを計算することにします。

In [ ]:
# metric = evaluate.load("accuracy")
# model.eval()
# for batch_e in eval_dataloader:
#     batch_eval = {k: v.to(device) for k, v in batch_e.items()}
#     with torch.no_grad():
#         outputs_eval = model(**batch_eval)

#     logits = outputs_eval.logits
#     predictions = torch.argmax(logits, dim=-1)
#     metric.add_batch(predictions=predictions, references=batch["labels"])

# metric.compute()